!pip install xdggs 
!pip install healpy
!pip install -U zarr==2.18.4
!pip install numcodecs==0.13.1

In [1]:
import warnings
import numpy as np
import xarray as xr
import xdggs
import fsspec

warnings.filterwarnings("ignore")
xr.set_options(display_max_rows=27)

In [2]:
fs = fsspec.filesystem("http")
#url="/home/jovyan/Akathon-Geilo/odysea_10_nest.zarr"
#url="https://data-fair2adapt.ifremer.fr/odysea/odysea.zarr"
url="https://data-fair2adapt.ifremer.fr/gebco/healpix_level_11.zarr"

ds=xr.open_zarr(url).pipe(xdggs.decode)
cell_centers = ds.dggs.cell_centers()
ds=ds.assign_coords(
    cell_centers.coords
)

In [3]:
lat = np.arange(56, 71, 0.01)
lon = np.arange(0, 15, 0.01)
full_lat = np.repeat(lat, len(lon))
full_lon = np.tile(lon, len(lat))

In [4]:
%%time
da = ds.gebco.dggs.sel_latlon(longitude=full_lon, latitude=full_lat).compute()

CPU times: user 4.97 s, sys: 2.52 s, total: 7.5 s
Wall time: 8.01 s


In [5]:
import cmocean
vmn = da.min()
vmx = da.max()
cmap = cmocean.tools.crop(cmocean.cm.topo, vmin=vmn, vmax=vmx, pivot=0)
da.dggs.explore(cmap=cmap)

Map(layers=[SolidPolygonLayer(filled=True, get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x7ea7b403…

In [6]:
#cmocean.cm.topo includes cmocean.cm.deep for values below pivot. You can also use cmocean.cm.deep for everything. 